In [ ]:
import threading
import time
import queue
from collections import defaultdict, deque

# Task object
class Task:
    def __init__(self, task_id, priority, processing_time):
        self.task_id = task_id
        self.priority = priority
        self.processing_time = processing_time
        self.remaining_time = processing_time

    def __repr__(self):
        return f"Task(id={self.task_id}, priority={self.priority}, remaining={self.remaining_time})"

# Task Scheduler class implementation
class TaskScheduler:
    def __init__(self, time_slice=1, num_workers=2):
        self.time_slice = time_slice
        self.task_queues = defaultdict(deque)  # priority -> deque of tasks
        self.lock = threading.Lock()
        self.running = True
        self.completed_tasks = []
        self.monitor_thread = threading.Thread(target=self.monitor)
        self.workers = [threading.Thread(target=self.worker_loop) for _ in range(num_workers)]

    # adding task in priority queue
    def add_task(self, task):
        with self.lock:
            self.task_queues[task.priority].append(task)
            print(f"[ADD] {task}")

    # return the highest priority task in FIFO order
    def get_next_task(self):
        # synchronization - to avoid any race condition
        with self.lock:
            for priority in sorted(self.task_queues.keys()):
                if self.task_queues[priority]:
                    # FIFO
                    task = self.task_queues[priority].popleft() 
                    return task
            return None

    # context switching 
    def requeue_task(self, task):
        # synchronization - to avoid any race condition
        with self.lock:
            self.task_queues[task.priority].append(task)

    # running queue + ready queue
    def worker_loop(self):
        while self.running:
            task = self.get_next_task()
            if task:
                exec_time = min(self.time_slice, task.remaining_time)
                print(f"[EXEC] {task.task_id} for {exec_time}s")
                time.sleep(exec_time)
                task.remaining_time -= exec_time

                # process execution done
                if task.remaining_time <= 0:
                    print(f"[DONE] {task.task_id}")
                    with self.lock:
                        self.completed_tasks.append(task)
                # context switching
                else:
                    self.requeue_task(task)
            else:
                time.sleep(0.1)  # Idle wait if no task is available


    # monitoring the task status asynchronously
    def monitor(self):
        while self.running:
            # synchronization - to avoid any race condition
            with self.lock:
                queue_status = {p: len(q) for p, q in self.task_queues.items()}
                print(f"[MONITOR] Queue sizes: {queue_status}, Completed: {len(self.completed_tasks)}")
            time.sleep(2)

    # starting monitor thread and both workers thread to run asynchronously
    def start(self):
        self.monitor_thread.start()
        for w in self.workers:
            w.start()

    # stopping the main thread until all workers threads and monitor thread executes completely
    def stop(self):
        self.running = False
        for w in self.workers:
            w.join()
        self.monitor_thread.join()

# main thread -  Round Robin(RR) CPU Scheduling
if __name__ == "__main__":
    scheduler = TaskScheduler(time_slice=1, num_workers=2)
    scheduler.start()

    scheduler.add_task(Task("T1", priority=2, processing_time=5))
    scheduler.add_task(Task("T2", priority=1, processing_time=3))
    scheduler.add_task(Task("T3", priority=2, processing_time=4))
    scheduler.add_task(Task("T4", priority=1, processing_time=6))
    scheduler.add_task(Task("T5", priority=3, processing_time=2))

    time.sleep(15)
    scheduler.stop()


[MONITOR] Queue sizes: {}, Completed: 0
[ADD] Task(id=T1, priority=2, remaining=5)
[ADD] Task(id=T2, priority=1, remaining=3)
[ADD] Task(id=T3, priority=2, remaining=4)
[ADD] Task(id=T4, priority=1, remaining=6)
[ADD] Task(id=T5, priority=3, remaining=2)
[EXEC] T2 for 1s
[EXEC] T4 for 1s
[EXEC] T2 for 1s
[EXEC] T4 for 1s
[MONITOR] Queue sizes: {2: 2, 1: 0, 3: 1}, Completed: 0
[EXEC] T2 for 1s
[EXEC] T4 for 1s
[DONE] T2
[EXEC] T1 for 1s
[EXEC] T4 for 1s
[MONITOR] Queue sizes: {2: 1, 1: 0, 3: 1}, Completed: 1
[EXEC] T3 for 1s
[EXEC] T4 for 1s
[EXEC] T1 for 1s
[EXEC] T4 for 1s
[MONITOR] Queue sizes: {2: 1, 1: 0, 3: 1}, Completed: 1
[EXEC] T3 for 1s
[DONE] T4
[EXEC] T1 for 1s
[EXEC] T3 for 1s
[EXEC] T1 for 1s
[MONITOR] Queue sizes: {2: 0, 1: 0, 3: 1}, Completed: 2
[EXEC] T3 for 1s
[EXEC] T1 for 1s
[DONE] T3
[EXEC] T5 for 1s
[DONE] T1
[MONITOR] Queue sizes: {2: 0, 1: 0, 3: 0}, Completed: 4
[EXEC] T5 for 1s
[DONE] T5
[MONITOR] Queue sizes: {2: 0, 1: 0, 3: 0}, Completed: 5
[MONITOR] Queue siz